In [0]:
import dlt
import requests
import json
from datetime import datetime
from src.api.endpoints import FIXTURE_ENDPOINT
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from src.api.api_handler import APIError, APIRequestHandler
from src.schemas.fields import TableNames
from src.utils.football_utils import FootballUtils
import concurrent.futures
from itertools import chain

In [0]:
spark.sql("USE SCHEMA `staging`")
rootUrl = dbutils.secrets.get(scope="football-analyze", key="api-url")
api_key = dbutils.secrets.get(scope="football-analyze", key="api-key")

# Define headers
headers = {
    'x-rapidapi-host': api_host,
    'x-rapidapi-key': api_key
}

REQUESTS_PER_MINUTE = 10
DELAY = 60 / REQUESTS_PER_MINUTE  # 6 seconds between requests

def fetch_fixtures(league_id, season=2023):
    with APIRequestHandler(base_url=rootUrl) as api:
        try:
            response = api.fetch_with_rate_limit(
                DELAY,
                headers,
                FIXTURE_ENDPOINT,
                params={'league': league_id, 'season': season}
            )
            fixtures_data = response.get('response', [])
            return fixtures_data
        except APIError as e:
            raise Exception("API Error: ", e)
        except Exception as e:
            raise Exception("Unexpected error: ", e)
